# Overview

In this notebook we will cover the basic concepts of inference on a trained U-net in Tensorflow. We will be specifically loading a network trained to perform segmentation of brain tumors from multimodal MR images. The data that we will be using in this tutorial comes from the MICCAI Brain Tumor Segmentation Challenge (BRaTS). More information about he BRaTS Challenge can be found here: http://braintumorsegmentation.org/

For basics of Tensorflow operation, neural networks and training, consider reviewing the preceding lectures in this series:

&nbsp;&nbsp;&nbsp;&nbsp; **01 - Introduction to Data, Tensorflow and Deep Learning** <br/>
&nbsp;&nbsp;&nbsp;&nbsp; **02 - Training a Classifier** <br/>
&nbsp;&nbsp;&nbsp;&nbsp; **03 - Inference with a Classifier**<br/>
&nbsp;&nbsp;&nbsp;&nbsp; **04 - Training a U-Net**

**Note:** Before running this notebook, be sure turn off the kernel used for training. You can keep the corresponding tab open in your browser to retrain later, just shut down the kernel from the top menu now so that it's resources can be used here (`Kernel` > `Shutdown`).

### Importing modules

To visualize inference for our simple classifer implementation, we will require two open-source libraries (`tensorflow`, `numpy`) as well as our custom modules created for this tutorial (`utils`, `data`). 

In [ ]:
import tensorflow as tf, numpy as np
import data
from utils import imshow

data.set_root(loc='/data/brats/npy')

### Loading trained model

Our trained model has been saved in the `../exp_unet/checkpoint` direction. Alternatively, you can also choose instead to load a pretrained model located in `../pretrained/unet`. Based on your preference, set the model dir into the `model_dir` variable. Then to restore this model we create a new `tf.Session` class and the `tf.train.import_meta_graph()` method to load the graph and parameters into memory.

In [ ]:
model_dir = '../exp_unet/checkpoint'
model_dir = '../pretrained/unet'

tf.reset_default_graph()
sess = tf.InteractiveSession()
saver = tf.train.import_meta_graph("%s/model.ckpy.meta" % model_dir)
saver.restore(sess, "%s/model.ckpy" % model_dir)

Using the named Graph collections specified during the training process, we can quickly reload handles to the key placeholders of our graph:

In [ ]:
X, mode, mask = tf.get_collection("inputs")
pred = tf.get_collection("outputs")[0]

### Running inference

Now that the graph structure is restored, we simply use the same `sess.run()` method to pass new data into their respective placeholders. As before, we will use the `data.load()` method to load new instances of validation data. Feel free to re-run the following cell repeatedly (ctrl + Enter) to see the network in action:

In [ ]:
dat, lbl, msk = data.load(mode='train', n=1, return_mask=True)
argmax = sess.run(pred, {X: dat, mode: False})
argmax = np.argmax(argmax, axis=3)
argmax[msk[..., 0] == 0] = 0

imshow(dat[..., 1], lbl=lbl if lbl.any() else None, title='FLAIR (ground-truth)')
imshow(dat[..., 1], lbl=argmax if argmax.any() else None, title='FLAIR (prediction)')